In [1]:
import h5py
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# set path
path = "/home/setholinger/Documents/Projects/PIG/detections/templateMatch/multiTemplate/run3/"

# read in backazimuths for catalog
azimuth_file = h5py.File(path + "event_backazimuths.h5","r")
event_backazimuths = np.array(list(azimuth_file["weighted_circular_average_event_backazimuths"]))
azimuth_file.close()

# read clustering results to get dispersive events
num_clusters = 2
cluster = 1
clust_file = h5py.File(path + "short_3D_clustering/modified_k_shape/" + str(num_clusters) + "/" + str(num_clusters) + "_cluster_predictions_0.05-1Hz.h5","r")
pred = np.array(list(clust_file["cluster_index"]))
clust_file.close()

# read detection times
time_file = h5py.File(path + "detection_times.h5","r")
det_times = np.array(list(time_file["times"]))
time_file.close()

In [3]:
# set baz range for rift, shear and mislocated events
rift_baz = [150,290]
shear_baz = [290,30+360]
flip_baz = [30,150]

In [4]:
# get backazimuths for cluster of dispersive events
cluster_backazimuths = event_backazimuths[pred==cluster]
cluster_times = det_times[pred==cluster]

In [5]:
# add 180 degrees to misloacted "east" events
for i in range(len(cluster_backazimuths)):
    if cluster_backazimuths[i] > flip_baz[0] and cluster_backazimuths[i] < flip_baz[1]:
        cluster_backazimuths[i] = cluster_backazimuths[i] + 180

In [6]:
# separate rift and shear margin events, saving event backazimuths and event times
rift_times = []
rift_backazimuths = []
shear_times = []
shear_backazimuths = []
nan_backazimuths = []
nan_times = []
for i in range(len(cluster_times)):
    if cluster_backazimuths[i] > rift_baz[0] and cluster_backazimuths[i] < rift_baz[1]:
        rift_times.append(cluster_times[i])
        rift_backazimuths.append(cluster_backazimuths[i])
    elif cluster_backazimuths[i] > shear_baz[0] and cluster_backazimuths[i] < shear_baz[1]:
        shear_times.append(cluster_times[i])
        shear_backazimuths.append(cluster_backazimuths[i])
    elif cluster_backazimuths[i] + 360 < shear_baz[1]:
        shear_times.append(cluster_times[i])
        shear_backazimuths.append(cluster_backazimuths[i])
    elif np.isnan(cluster_backazimuths[i]) == 1:
        nan_times.append(cluster_times[i])
        nan_backazimuths.append(cluster_backazimuths[i])

In [9]:
# save results into h5 file
out_file = h5py.File(path + "short_3D_clustering/modified_k_shape/" + str(num_clusters) + "/rift_and_shear_backazimuths.h5","a")
out_file.create_dataset("rift_backazimuths",data=rift_backazimuths)
out_file.create_dataset("rift_times",data=rift_times)
out_file.create_dataset("shear_backazimuths",data=shear_backazimuths)
out_file.create_dataset("shear_times",data=shear_times)
out_file.create_dataset("nan_backazimuths",data=shear_backazimuths)
out_file.create_dataset("nan_times",data=shear_times)
out_file.close()